# How to use BulkLMM scan from BigRiverQTL in R with JuliaConnectoR?

`JuliaConnectoR` is a package that enables the integration of Julia language into R programming. 
With `JuliaConnectoR`, users can execute Julia commands and functions within R, utilizing Julia's 
high-performance computing capabilities while still working within the familiar R environment. 
The package provides seamless communication between the two languages, allowing for data to be 
passed between R and Julia with ease. `JuliaConnectoR` also provides for the installation of Julia 
packages from within R, making it a convenient tool for those looking to leverage the power 
of Julia while still utilizing R as their primary programming language.   

*Note: [Link to the documentation of the package `JuliaConnectoR`](https://github.com/stefan-m-lenz/JuliaConnectoR?tab=readme-ov-file).*

## Environment and libraries

In [1]:
if (!require(JuliaConnectoR)) {
  install.packages("JuliaConnectoR")
}
if (!require(png)) {
  install.packages("magick")
}

Loading required package: JuliaConnectoR

Loading required package: png



In [2]:
# load necessary library
library(JuliaConnectoR)
library(magick)


Linking to ImageMagick 6.9.13.29
Enabled features: cairo, freetype, fftw, ghostscript, heic, lcms, pango, raw, rsvg, webp
Disabled features: fontconfig, x11



Create a Julia project environment:

In [3]:
juliaEval('using Pkg')
juliaEval('Pkg.activate(".")')

Starting Julia ...



Install `BigRiverQTL` and `Plots` if it is not already installed in Julia:

In [4]:
# Change need2install into TRUE if you need to install the Julia libraries
need2install <- FALSE
if(need2install){
    juliaEval('Pkg.add("BigRiverQTL")')
}

In [5]:
# Change need2install into TRUE if you need to install the Julia libraries
need2install <- FALSE
if(need2install){
    juliaEval('Pkg.add("Plots")')
}

Load the library `BigRiverQTL`and `Plots`:

In [6]:
 juliaEval('using BigRiverQTL, Plots')

Load the `BigRiverQTL` package so that its functions can be accessed from within R.

In [7]:
# load BigRiverQTL features available in R
BigRiverQTL <- juliaImport("BigRiverQTL")

## Example - BXD 

### Data

In [8]:
########
# Data #
########
# get path of data example from the package
pathBigRiverQTL <- juliaEval('pathof(BigRiverQTL) |> dirname |>dirname')
data_path <- file.path(pathBigRiverQTL, "test", "data", "BXD", "bxd.json")  

# check if path is correct
data_path |> file.exists()

[1] TRUE

Load bxd data using the function `get_geneticstudydata()`: 

In [9]:
# Load bxd data
data <- BigRiverQTL$get_geneticstudydata(data_path)

The current version of `BigRiverQTL` does not have imputation functions.

In [10]:
# The current version of `BigRiverQTL` does not have imputation functions.
# Remove the  missing data
data <- BigRiverQTL$get_data_completecases(data);

Get the markers info:

In [11]:
# gmap contains makers info 
gInfo <- data$gmap;

In [12]:
juliaGet(gInfo) |> names()

[1] "chr"         "marker_name" "pos"         "unit"

Get the phenotype info:

In [13]:
# phenotype info 
pInfo <- data$phenocov;

In [14]:
juliaGet(pInfo) |> names()

[1] "traits"       "descriptions"

In [15]:
# phenotype values 
pheno <- data$pheno$val;

Get the genotype matrix:

In [16]:
# We can get the genotype matrix using the following command.
# For computing reasons, we need to convert the geno matrix in Float64.
# One way to do it is to multiply by 1.0
geno <- juliaLet("reduce(hcat, genoval).*1.0", genoval = data$geno$val)

### Preprocessing

In [17]:
#################
# Preprocessing #
#################
traitID <- 868;
pheno_y <- pheno[, traitID];
idx_not_missing <- which(!is.na(pheno_y));

### Kinship

In [18]:
###########
# Kinship #
###########
kinship <- BigRiverQTL$kinship_gs(geno[idx_not_missing,],0.99);

### Scan

In [19]:
########
# Scan #
########

single_results_perms <- BigRiverQTL$scan(
	pheno_y[idx_not_missing],
	geno[idx_not_missing,],
	kinship,
	permutation_test = TRUE,
	nperms = 1000L
);

### Plots

In [20]:
#########
# Plots #
#########

# QTL plots
p <- BigRiverQTL$plot_QTL(single_results_perms, gInfo, mbColname = "Pos", dpi = 600L)
juliaLet('savefig(p, "qtl_plot.png");', p = p)

[1] "C:\\git\\gregfa\\bigriver\\BigRiverQTL.jl\\example\\R\\qtl_plot.png"

In [81]:
# image_read("qtl_plot.png") |> image_scale("600") |> print()

<img src="qtl_plot.png" alt="drawing" width="500" />

We can also display directly the figure from Julia:

In [21]:
juliaLet('display(p)', p = p)

In [22]:
# Manhattan plots
p <- BigRiverQTL$plot_manhattan(single_results_perms, gInfo, mbColname = "Pos", dpi = 660L)
juliaLet('savefig(p, "manhattan_plot.png");', p = p);

[1] "C:\\git\\gregfa\\bigriver\\BigRiverQTL.jl\\example\\R\\manhattan_plot.png"

In [23]:
juliaLet('display(p)', p = p)

<img src="manhattan_plot.png" alt="drawing" width="500" />